# シミュレーション
## 概要
本ノートブックは修士論文の研究で用いたシミュレーション研究の一部をRのプログラムとともにここに述べる．一般化線形混合効果モデルを用いた分析のコード例は別のノートブックで取り扱う．

## 目的
本シミュレーション実験の目的は，マルチレベル多重補完法（欠測補完として知られる多重補完法をマルチレベルに応用したもの）を適用して得たパラメータ推定値のバイアスを CC 解析や通常の多重補完法のバイアスと比較することである．マルチレベルモデルとしてレベル$1$を個体内の繰り返しレベル，レベル$2$を個体レベルと定義する．


## データ
データセットはマルチレベルを仮定しており，$2$値アウトカム$y$と$4$つの連続量の変数$x_1,x_2,w_1,w_2$および変量効果$\gamma$の組 $(y, x_1, x_2, w_1, w_2, \gamma)$ を作成した．$x_1$と$w_1$はレベル$1$の変数，$x_2$と$w_2$はレベル$2$の変数であり，$x_1$と$x_2$はアウトカムの共変量に使用され，欠測が生じる．$w_1$, $w_2$はそれぞれ，$x_1$,$x_2$の補助変数 (auxiliary variable) であり，MARメカニズムにおいて欠測の説明に用い，アウトカムの解析モデルには含めない．

## ライブラリ読み込み

In [ ]:
library(dplyr) 
library(mice)
library(miceadds)

## 実行コード

患者サンプルサイズ$100$とし、患者の反復回数を一律$10$としたデータセットを考える．
アウトカムをイベント発生率が$40%$になるように，次のモデル式を用いて，完全データを作成した．
ここで変量効果$\gamma$の分散は$\sqrt{2}$とした．
コードではシミュレーション回数を$5$としているが$500$回で行った．

$$
logit(Y) = −0.51 + 1 \cdot x_1 + 1 \cdot x_2 + \gamma
$$

In [ ]:
# モデルIDと作成日を付与
testdate<-"0522"
target_folder <- 'Model'
# datasetの数（実際は500）
X<-5
# samplesize(患者)
N<-100

# 多項分布の場合は1、それ以外の値だと反復繰り返し
multinomialflg<-0
# clustersize（上で1のときは意味ない）
cluster_size <- 10

#イベントの発生に関わるロジスティック回帰式の回帰係数の決定
beta_intercept <- log(0.6)
beta_missing   <- 1

beta0<- beta_intercept
beta1<- beta_missing
beta2<- beta_missing

# 変量効果の分散の数
std<-sqrt(2)

# 変数間の相関（本当は分布決定のフラグも入れたい）

# x1,x2,w1,w2の生成ロジック
# 1:4変数正規分布, 
# 2:2変数×2変数の正規分布
pattern <- 1
setrho <- 0.6

setfolder<-function(foldername){
    exectime <- paste(strftime(Sys.Date(), format="%Y%m%d"),strftime(Sys.time(), format="%H%M%S"),sep="")
    to <- paste(foldername,"_",exectime,sep="")
    if(dir.exists(foldername)==TRUE){
      file.rename(from=foldername, to=to)
    }
    dir.create(foldername)
}

code_folder<-paste("00_",target_folder,sep="")
folder1    <-paste("11_",target_folder,"_mcar",sep="")
folder2    <-paste("21_",target_folder,"_mar",sep="")
folder3    <-paste("31_",target_folder,"_mnar",sep="")

setfolder(code_folder)
setfolder(folder1)
setfolder(folder2)
setfolder(folder3)

# ===== 関数の定義 ======
logistic <- function(x) exp(x) / (1 + exp(x))

# クラスタサイズでx1,x2,x3,x4作る

# 関数の定義
# namedはc("x1","w1")を想定
create_basedf2 <- function(clustersize,named){
    # 2変数の正規分布を作成する．
    # 共分散構造
    mu <- c(0,0)
    var   <- 1
    rho   <- setrho
    Sigma <- matrix(c(var ,rho ,
                      rho ,var ),nrow=2)     
    base <- MASS::mvrnorm(n=clustersize,mu,Sigma=Sigma)
    base_df <- data.frame(base)
    names(base_df) <- named
    base_df
}

loops <- function(df){
    patient_idv= df[[1]]
    x2_v = df[[2]]
    w2_v = df[[3]]
    gamma_v = df[[4]]
    patient_id = rep(patient_idv,cluster_size)
    gamma = rep(gamma_v,cluster_size)
    x2_col = rep(x2_v,cluster_size)
    w2_col = rep(w2_v,cluster_size)
    mid_df <- create_basedf2(cluster_size,c("x1","w1"))
    ret_df <- data.frame(patient_id,mid_df,x2=x2_col,w2=w2_col,gamma)
    return (ret_df)
}

execute2<-function(){
    # 新コード
    patients<-seq(1,N)
    gamma <- rnorm(n=N,mean=0,sd=sqrt(2))
    df1 <- create_basedf2(N,c("x2","w2"))
    df2 <- data.frame(patients,df1,gamma)
    
    ret<-apply(df2,1,loops)
    
    base_df <- matrix(,nrow = 0, ncol = 6)
    for(tmp in ret){
        base_df <- rbind(base_df,tmp)
    }
    outcome <- apply(base_df,1,setoutcome)
    out_df<-data.frame(outcome,base_df)
    return (out_df)
}

create_basedf <- function(size){
    # 平均
    mu <- c(0,0,0,0)
    # 共分散構造
    var   <- 1
    rho   <- setrho
    Sigma <- matrix(c(var ,rho ,rho ,rho ,
                      rho ,var ,rho ,rho ,
                      rho ,rho ,var ,rho ,
                      rho ,rho ,rho ,var   
                    ),nrow=4) 
    
    # d02はdの初期ステップの2番目という意味
    d02 <- MASS::mvrnorm(n=size,mu,Sigma=Sigma)
    named <- c("x1","w1","x2","w2")
    names(d02) <- named    
    df<-data.frame(x1=d02[,1],w1=d02[,2],x2=d02[,3],w2=d02[,4])    
}


setoutcome<-function(x){
    x1=x["x1"]
    x2=x["x2"] 
    gamma=x["gamma"]
    pi=logistic(beta0+beta1*x1+beta2*x2+gamma)
    outcome_row = rbinom(1,1,pi)
}

# 100人の分布
cluster_distribution<-function(){
    if(multinomialflg==1){
        d01 <- rmultinom(n=N, size = 1, prob = c(0.3,0.4,0.3))
        repeated_arr<-1*d01[1,]+2*d01[2,]+3*d01[3,]
    } else {
        repeated_arr<-rep(cluster_size,N)
    }
}

#実際
execute<-function(){
    # 分布確定
    repeated_arr<-cluster_distribution()
    total_rows<-sum(repeated_arr)
    # 患者番号の付与
    patients <- c()
    index <- 0
    for(patient_repeated in repeated_arr){
        index <- index + 1
        add <- rep(index,patient_repeated)
        patients <- c(patients, add)
    }
    # 多変数正規分布よりデータを生成
    df1 <- create_basedf(total_rows)
    df1_2 <- data.frame(patient_id=patients,df1)

    # groupingして
    df2 <- df1_2 %>%
        group_by(patient_id) %>%
        slice_head(n = 1)
    # gammaをつなげる。
    gamma <- rnorm(n=N,mean=0,sd=std)
    df2_2 <- data.frame(df2,gamma)

    df3 <-dplyr::inner_join(df1_2 , df2_2 ,by="patient_id") %>% 
    select(patient_id=patient_id,x1=x1.x,w1=w1.x,x2=x2.y,w2=w2.y,gamma=gamma) 
    outcome <- apply(df3,1,setoutcome)
    out_df<-data.frame(outcome,df3)
    return (out_df)
}

# Xこのデータ生成
setup<-function(){
  measured = c()
  for(loop in seq(1,X)){
    counter <- sprintf("%03d", loop)
    df4 <- data.frame()
　  # 切り替えをフラグにしたいね。
    if(pattern==1){
      df4 <- execute()
    } else if(pattern == 2){
      df4 <- execute2()
    } else {
      #後続の処理しない
      return(0)
    }
    measured <- c(measured,mean(df4$outcome))
    # ファイル名の生成
    filepath<-paste(code_folder,"\\exportdata_",testdate,"_",counter,".csv",sep="")
    write.csv(x = df4, file = filepath,row.names = FALSE)
  }
  # アウトカムの生成割合
  print(paste("mean:", mean(measured)))
  print(paste("var:", var(measured)))
}

setup()

## マルチレベルの欠測値生成シミュレーション
$2$つの変数の欠測の割合は $30%$とし，欠測発生のモデル式をメカニズムごとに以下の
ように定義した．

In [ ]:
# Level1の欠測生成
setx1<-function(data){
    x1=data[["x1"]]
    w1=data[["w1"]]
    
    x1_mar_beta0 <- 1.1
    x1_mar_beta1 <- 1.5
        
    x1_mnar_beta0 <- 1
    x1_mnar_beta1 <- 0.6
    
    x1.mar  <- 1 - rbinom(1,1,logistic(x1_mar_beta0 + x1_mar_beta1 * w1))   
    x1.mnar <- 1 - rbinom(1,1,logistic(x1_mnar_beta0 + x1_mnar_beta1 * x1))
    df = data.frame(x1.mar,x1.mnar)
}

# Level2の欠測生成
setx2<-function(data){
    patient_id = data[["patient_id"]]
    x2=data[["x2"]]
    w2=data[["w2"]]
    
    x2_mar_beta0 <- 1.2
    x2_mar_beta1 <- 1.4
    
    x2_mnar_beta0 <- 1.0
    x2_mnar_beta1 <- 0.6
 
    x2.mar  <- 1 - rbinom(1,1,logistic(x2_mar_beta0  + x2_mar_beta1  * w2))  
    x2.mnar <- 1 - rbinom(1,1,logistic(x2_mnar_beta0 + x2_mnar_beta1 * x2))
    
    df = data.frame(patient_id,x2.mar,x2.mnar)
}

# ファイル読み込み
readfile<-function(counter){
    filepath <- filepath<-paste(code_folder,"\\exportdata_",testdate,"_",counter,".csv",sep="")
    data<-read.csv(filepath)
    return (data)
}

logistic <- function(x) exp(x) / (1 + exp(x))

measure1 = c()
measure2 = c()
measure3 = c()
measure4 = c()
measure5 = c()
measure6 = c()

for(loop in seq(1,X)){
  
  # ファイル読み込み
  counter <- sprintf("%03d", loop)
  data <- readfile(counter)

  # Lv2のデータを抽出する。
  data2 <- select(.data = data, patient_id,x2,w2)　%>% distinct()
　# x3のセット
  ret <- apply(data2,1,setx2)
  data_x2flg <- matrix(,nrow = 0, ncol = 3)
  colnames(data_x2flg) <- c("patient_id","x2.mar","x2.mnar")
  for(tmp in ret){
    data_x2flg <- rbind(data_x2flg,tmp)
  }
  data3 <- inner_join(data, data_x2flg ,by="patient_id")

  # x1のセット
  ret <- apply(data,1,setx1)  
  df_x1flg <- matrix(,nrow = 0, ncol = 2)
  colnames(df_x1flg) <- c("x1.mar","x1.mnar")
  for(tmp in ret){
    df_x1flg <- rbind(df_x1flg,tmp)
  }
  data4<-data.frame(data3,df_x1flg)

  # MCARのセット    
  Size <- sum(count(data))
  num_of_patient <- sum(count(data2))
  x1.mcar<-rbinom(Size, 1, 0.3)
  x2.mcar<-rbinom(num_of_patient, 1, 0.3)
  data_x2mcar <- data.frame(patient_id=seq(1,num_of_patient),x2.mcar)
  data5 <- inner_join(data4, data_x2mcar ,by="patient_id")
  data6 <- data.frame(data5,x1.mcar)
    
  x1.mcar<-ifelse(data6$x1.mcar==1,NA,data6$x1)
  x2.mcar<-ifelse(data6$x2.mcar==1,NA,data6$x2)

  x1.mar<-ifelse(data6$x1.mar==1,NA,data6$x1)
  x2.mar<-ifelse(data6$x2.mar==1,NA,data6$x2)

  x1.mnar<-ifelse(data6$x1.mnar==1,NA,data6$x1)
  x2.mnar<-ifelse(data6$x2.mnar==1,NA,data6$x2)

  data7.mcar <- data.frame(
      outcome = data6$outcome,
      patient_id = data6$patient_id,
      x1 = x1.mcar,
      w1 = data6$w1,
      x2 = x2.mcar,
      w2 = data6$w2)    

  data7.mar <- data.frame(
      outcome = data6$outcome,
      patient_id = data6$patient_id,
      x1 = x1.mar,
      w1 = data6$w1,
      x2 = x2.mar,
      w2 = data6$w2)    
      
  data7.mnar <- data.frame(
      outcome = data6$outcome,
      patient_id = data6$patient_id,
      x1 = x1.mnar,
      w1 = data6$w1,
      x2 = x2.mnar,
      w2 = data6$w2)    

    cnt = count(data)

    x1<-(sum(is.na(x1.mcar))/cnt)
    measure1 <- c(measure1,x1$n)
    
    x2<-(sum(is.na(x1.mar))/cnt)
    measure2 <- c(measure2,x2$n)

    x3<-(sum(is.na(x1.mnar))/cnt)
    measure3 <- c(measure3,x3$n)
    
    
    x4<-(sum(is.na(x2.mcar))/cnt)
    measure4 <- c(measure4,x4$n)
    
    x5<-(sum(is.na(x2.mar))/cnt)
    measure5 <- c(measure5,x5$n)

    x6<-(sum(is.na(x2.mnar))/cnt)
    measure6 <- c(measure6,x6$n)
    
    fp1=paste("11_",target_folder,"_mcar\\",counter,".csv",sep="")
    fp2=paste("21_",target_folder,"_mar\\" ,counter,".csv",sep="")
    fp3=paste("31_",target_folder,"_mnar\\",counter,".csv",sep="")
    
    write.csv(x = data7.mcar, file = fp1,row.names = FALSE)
    write.csv(x = data7.mar,  file = fp2,row.names = FALSE)
    write.csv(x = data7.mnar, file = fp3,row.names = FALSE)
}

# MCAR Level1の欠測率評価
print(mean(measure1))
print(var(measure1))

# MAR Level1の欠測率評価
print(mean(measure2))
print(var(measure2))

# MNAR Level1の欠測率評価
print(mean(measure3))
print(var(measure3))

# MNAR Level1の欠測率評価
print(mean(measure4))
print(var(measure4))

# MNAR Level2の欠測率評価
print(mean(measure5))
print(var(measure5))

# MNAR Level2の欠測率評価
print(mean(measure6))
print(var(measure6))

## 欠測値補完
miceを用いて、欠測値を補完する。

In [ ]:
imputation_singlelevel <- function(data){
  impMethod <- character(ncol(data))
  names(impMethod) <- colnames(data)
  impMethod['x1'] <- "pmm"
  impMethod['x2'] <- "pmm"
    
  predMatrix <- matrix(0, ncol(data), ncol(data))
  rownames(predMatrix) <- colnames(predMatrix) <- colnames(data) 
  
  predMatrix["outcome",]     <- c(0, 0,1,1,1,1)
  predMatrix["patient_id",]  <- c(0, 0,0,0,0,0)
  predMatrix["x1",]          <- c(1, 0,0,1,1,1)
  predMatrix["w1",]          <- c(1, 0,1,0,1,1)
  predMatrix["x2",]          <- c(1, 0,1,1,0,1) 
  predMatrix["w2",]          <- c(1, 0,1,1,1,0)
    
  imputation_singlelevel <- mice(data, pred = predMatrix, method = impMethod, m = 5, maxit = 1,print = TRUE, seed = 1234567)
}

imputation_multilevel <- function(data){

  impMethod <- character(ncol(data))
  names(impMethod) <- colnames(data)
  impMethod['x1'] <- "2l.pmm"
  impMethod['x2'] <- "2lonly.norm"
  print(impMethod)
  predMatrix <- matrix(0, ncol(data), ncol(data)) 
  rownames(predMatrix) <- colnames(predMatrix) <- colnames(data) 

  predMatrix["outcome",]     <- c(0,-2,1,1,1,1)
  predMatrix["patient_id",]  <- c(1, 0,1,1,1,1)
  predMatrix["x1",]          <- c(1,-2,0,1,1,1)
  predMatrix["w1",]          <- c(1,-2,1,0,1,1)
  predMatrix["x2",]          <- c(1,-2,1,1,0,1) 
  predMatrix["w2",]          <- c(1,-2,1,1,1,0)
    
  imp_multilevel <- mice(data, pred = predMatrix, method = impMethod, m = 5, maxit = 1,print = TRUE, seed = 1234567)
}

read_write<-function(in_filepath,out1,out2){
  data<-read.csv(in_filepath)
  data<- data[, colnames(data) != "gamma"]
  imputation_singlelevel<- imputation_singlelevel(data)
  write.mice.imputation(mi.res=imputation_singlelevel, name=out_1,
                      include.varnames=TRUE,
                      long=FALSE, mids2spss=FALSE,
                      spss.dec=",", dattype=NULL)
  
  data<- data[, colnames(data) != "gamma"]
    
  imp_multilevel <- imputation_multilevel(data)
  write.mice.imputation(mi.res=imp_multilevel, name=out_2,
                      include.varnames=TRUE,
                      long=FALSE, mids2spss=FALSE,
                      spss.dec=",", dattype=NULL)
}

for(i in 1:X){
  # ファイル読み込み
　cross_str <- sprintf("%03d", i)
  in_filepath<- paste(".\\11_",target_folder,"_mcar\\",cross_str,".csv",sep="")  
  out_1 <- paste("12_",target_folder,"mcar_imp_",cross_str,sep="")
  out_2 <- paste("13_",target_folder,"mcar_mlimp_",cross_str,sep="")
    
  read_write(in_filepath,out_1,out_2)
    
  in_filepath<- paste("21_",target_folder,"_mar\\",cross_str,".csv",sep="")  
  out_1 <- paste("22_",target_folder,"mar_imp_",cross_str,sep="")
  out_2 <- paste("23_",target_folder,"mar_mlimp_",cross_str,sep="")
      
  read_write(in_filepath,out_1,out_2)
    
  in_filepath<- paste("31_",target_folder,"_mnar\\",cross_str,".csv",sep="")  
  out_1 <- paste("32_",target_folder,"mnar_imp_",cross_str,sep="")
  out_2 <- paste("33_",target_folder,"mnar_mlimp_",cross_str,sep="")

  read_write(in_filepath,out_1,out_2)
}